In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("tawsifurrahman/tuberculosis-tb-chest-xray-dataset")

print("Path to dataset files:", path)

Using Colab cache for faster access to the 'tuberculosis-tb-chest-xray-dataset' dataset.
Path to dataset files: /kaggle/input/tuberculosis-tb-chest-xray-dataset


Segmentasi Paru dan Lesi

In [ ]:
import os
import cv2
import numpy as np
import glob
from tqdm import tqdm
from sklearn.cluster import KMeans
from skimage import filters, morphology, measure, exposure, segmentation, feature
from scipy.ndimage import binary_fill_holes
from skimage.measure import regionprops

# Segmentasi Paru

# Preprocessing citra menggunakan AHE
def img_preprocess(img):
    img_u8 = (img * 255).astype(np.uint8)
    denoised = cv2.bilateralFilter(img_u8, 9, 75, 75)
    img_clean = denoised.astype(np.float32) / 255.0
    img_enhanced = exposure.equalize_adapthist(img_clean, clip_limit=0.02)
    return img_enhanced

# membuat mask tubuh
def make_torso_mask(img):
    try: thr = filters.threshold_otsu(img)
    except: return np.ones_like(img, dtype=np.uint8)
    mask = img > thr * 0.5
    mask = morphology.remove_small_objects(mask, 5000)
    mask = morphology.binary_closing(mask, morphology.disk(5))
    mask = binary_fill_holes(mask)
    h, w = mask.shape
    border_cut = int(w * 0.05)
    mask[:, :border_cut] = 0
    mask[:, -border_cut:] = 0
    return mask.astype(np.uint8)

# mensegmentasi paru pakai k-means clustering (ada di slide bapaknya)
def segment_kmeans(img, torso_mask):
    if np.sum(torso_mask) == 0: return np.zeros_like(img, dtype=np.uint8)
    masked_pixels = img[torso_mask > 0].reshape(-1, 1)
    if masked_pixels.shape[0] < 100: return np.zeros_like(img, dtype=np.uint8)
    kmeans = KMeans(n_clusters=2, random_state=42, n_init=5).fit(masked_pixels)
    centers = kmeans.cluster_centers_
    lung_cluster_label = np.argmin(centers)
    labels = kmeans.labels_
    full_labels = np.zeros(img.shape, dtype=int)
    full_labels[torso_mask > 0] = labels
    lung_mask = (full_labels == lung_cluster_label) & (torso_mask > 0)
    return lung_mask

# menentukan seed (titik awal) watershed
def get_clean_seed(raw_mask):
    v_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 25))
    v_seed = cv2.morphologyEx(raw_mask.astype(np.uint8), cv2.MORPH_OPEN, v_kernel)
    try: v_seed = morphology.remove_small_objects(v_seed.astype(bool), 1000)
    except: pass
    severed_mask = morphology.binary_erosion(raw_mask, morphology.disk(10))
    labels = measure.label(severed_mask)
    clean_seed = np.zeros_like(raw_mask, dtype=bool)
    for region in measure.regionprops(labels):
        region_mask = (labels == region.label)
        if np.any(region_mask & v_seed):
            clean_seed = clean_seed | region_mask
    return clean_seed

# menentukan batas paru
def get_barrier(img, torso_mask, clean_seed):
    if np.sum(torso_mask) == 0: return np.zeros_like(img)
    bright_thresh = np.percentile(img[torso_mask > 0], 60)
    bright_areas = (img > bright_thresh) & (torso_mask > 0)
    skeleton = morphology.skeletonize(bright_areas)
    img_u8 = (img * 255).astype(np.uint8)
    canny_edges = cv2.Canny(img_u8, 40, 120) > 0
    raw_barrier = (skeleton | canny_edges)
    internal_zone = morphology.binary_dilation(clean_seed, morphology.disk(25))
    final_barrier = raw_barrier & (~internal_zone)
    bg_outside = (torso_mask == 0)
    final_barrier = final_barrier | bg_outside
    return final_barrier

# menghaluskan mask yang kroak
def smoothen_mask(binary_mask):
    mask_u8 = (binary_mask * 255).astype(np.uint8)
    blurred = cv2.GaussianBlur(mask_u8, (25, 25), 0)
    _, smooth_mask = cv2.threshold(blurred, 127, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(smooth_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    final_canvas = np.zeros_like(mask_u8)
    contours = sorted(contours, key=cv2.contourArea, reverse=True)[:2]
    for cnt in contours:
        epsilon = 0.003 * cv2.arcLength(cnt, True)
        approx = cv2.approxPolyDP(cnt, epsilon, True)
        cv2.drawContours(final_canvas, [approx], -1, 255, -1)
    return final_canvas

# segmentasi paru dengan segala alat di atasnya
def segment_lungs(img_original):
    # Pipeline V26 Lengkap
    img_resized = cv2.resize(img_original, (512, 512))
    img_float = img_resized.astype(np.float32) / 255.0

    try:
        img_enhanced = img_preprocess(img_float)
        torso_mask = make_torso_mask(img_enhanced)
        raw_lung = segment_kmeans(img_enhanced, torso_mask)
        clean_seed = get_clean_seed(raw_lung)
        barrier_map = get_barrier(img_enhanced, torso_mask, clean_seed)

        img_smooth = cv2.GaussianBlur(img_enhanced, (3, 3), 0)
        elevation_map = filters.sobel(img_smooth)

        markers = np.zeros_like(raw_lung, dtype=np.int32)
        markers[clean_seed == 1] = 1
        markers[barrier_map == 1] = 2

        watershed_result = segmentation.watershed(elevation_map, markers)
        lung_watershed = (watershed_result == 1)
        filled_lung = binary_fill_holes(lung_watershed)

        # Convex Smoothing
        final_mask = np.zeros_like(raw_lung, dtype=np.uint8)
        labels = measure.label(filled_lung)
        props = sorted(measure.regionprops(labels), key=lambda x: x.area, reverse=True)[:2]

        for prop in props:
            single_lung = (labels == prop.label)
            chull = morphology.convex_hull_image(single_lung)
            reference_shape = morphology.binary_dilation(single_lung, morphology.disk(10))
            smoothed_lung = chull & reference_shape
            final_mask[smoothed_lung] = 1

        final_mask_smooth = smoothen_mask(final_mask)
        return final_mask_smooth, img_enhanced # Return img_enhanced untuk dipakai deteksi lesi

    except:
        return None, None

# Deteksi Lesi
def detect_lesions(img_enhanced, lung_mask):
    """
    V26 STRICT: Versi konservatif untuk mengurangi False Positive pada paru normal.
    Hanya menandai lesi jika BENAR-BENAR yakin (Sangat gelap/Sangat kasar).
    """
    masks = {}
    h, w = img_enhanced.shape

    # Pastikan mask binary
    lung_mask_bool = lung_mask > 0

    # Hitung statistik HANYA di dalam paru
    roi_pixels = img_enhanced[lung_mask_bool]
    if len(roi_pixels) == 0:
        return {k: np.zeros_like(lung_mask) for k in ['calcification', 'cavity', 'infiltrate', 'effusion']}

    mean_val = np.mean(roi_pixels)
    std_val = np.std(roi_pixels)

    # Deteksi Calcification (titik sangat terang dan kecil)
    thresh_calc = mean_val + 3.0 * std_val
    mask_calc = (img_enhanced > thresh_calc) & lung_mask_bool

    # Filter ukuran: Kalsifikasi itu bintik kecil, bukan area luas
    # Hapus yang terlalu besar (kemungkinan tulang) atau terlalu kecil (noise)
    mask_calc = morphology.remove_small_objects(mask_calc, 10) # Min 10 px

    # Hapus blob besar (misal > 500 px) karena itu pasti tulang
    lbl_calc = measure.label(mask_calc)
    for prop in regionprops(lbl_calc):
        if prop.area > 500:
            mask_calc[lbl_calc == prop.label] = 0

    masks['calcification'] = mask_calc

    # Deteksi Cacity (area gelap yang bulat)
    thresh_dark = mean_val - 1.5 * std_val

    # Fokus area atas paru saja (Top 60%)
    top_lung = lung_mask_bool.copy()
    top_lung[int(h*0.6):, :] = 0

    raw_cavity = (img_enhanced < thresh_dark) & top_lung

    # Filter Geometri: Cavity TB biasanya bulat/oval. Celah rusuk itu pipih.
    # Cek Eccentricity (Kelonjongan). 0=Bulat, 1=Garis.
    # Cek Solidity (Kepadatan).
    mask_cavity = np.zeros_like(lung_mask)
    lbl_cav = measure.label(raw_cavity)

    for prop in regionprops(lbl_cav):
        # Syarat: Area cukup besar, TIDAK terlalu lonjong (Eccentricity < 0.9), Padat (Solidity > 0.8)
        if prop.area > 150 and prop.eccentricity < 0.9 and prop.solidity > 0.85:
            mask_cavity[lbl_cav == prop.label] = 1

    masks['cavity'] = mask_cavity

    # Deteksi Infiltrate (tekstur sangat kasar)
    img_u8 = (img_enhanced * 255).astype(np.uint8)
    entropy_img = filters.rank.entropy(img_u8, morphology.disk(5), mask=lung_mask)

    # Threshold entropy (misal > 5.5). Normal rib cage sekitar 4.0-5.0.
    mask_texture = (entropy_img > 5.5) & lung_mask_bool

    # Hapus area tulang (calcification mask) dari sini
    mask_texture = mask_texture & (~mask_calc)

    # Clean up noise bintik
    mask_texture = morphology.binary_opening(mask_texture, morphology.disk(3))

    masks['infiltrate'] = mask_texture

    # Deteksi Effusion (cairan di sudut bawah)
    bottom_slice_h = int(h * 0.75)
    bottom_lung = lung_mask_bool.copy()
    bottom_lung[:bottom_slice_h, :] = 0

    effusion_final_mask = np.zeros_like(lung_mask)

    if np.sum(bottom_lung) > 0:
        # Pisahkan paru kiri dan kanan di area bawah
        lbl_bottom = measure.label(bottom_lung)

        for region in regionprops(lbl_bottom):
            # Abaikan noise kecil
            if region.area < 500: continue

            single_side_bottom = (lbl_bottom == region.label)

            # Hitung Convex Hull per sisi
            chull_side = morphology.convex_hull_image(single_side_bottom)

            # Kandidat = Selisih Hull dan Mask Asli
            candidate = chull_side & (~single_side_bottom)

            # FILTERING KETAT:
            # 1. Hapus area kecil (< 1500 px)
            candidate_big = morphology.remove_small_objects(candidate, 1500)

            # 2. Hapus garis tipis dengan Opening
            candidate_solid = morphology.binary_opening(candidate_big, morphology.disk(5))

            effusion_final_mask = effusion_final_mask | candidate_solid

        masks['effusion'] = effusion_final_mask
    else:
        masks['effusion'] = np.zeros_like(lung_mask)

    return masks

def create_multi_overlay(original, lung_mask, lesion_masks):
    """
    Membuat overlay warna-warni:
    - Paru: Ungu Transparan
    - Calcification: Kuning
    - Cavity: Merah
    - Infiltrate: Biru Cyan
    - Effusion: Hijau
    """
    if len(original.shape) == 2:
        vis = cv2.cvtColor(original, cv2.COLOR_GRAY2BGR)
    else:
        vis = original.copy()

    # 1. Overlay Paru Dasar (Ungu)
    lung_color = (255, 0, 255) # Magenta
    overlay_lung = vis.copy()
    overlay_lung[lung_mask > 0] = lung_color
    vis = cv2.addWeighted(overlay_lung, 0.3, vis, 0.7, 0)

    # 2. Overlay Lesi (Lebih Solid)
    # Urutan penting: Lesi kecil ditumpuk di atas lesi besar

    # Infiltrate (Cyan)
    if np.sum(lesion_masks['infiltrate']) > 0:
        vis[lesion_masks['infiltrate'] > 0] = (255, 255, 0) # Cyan (B,G,R)

    # Effusion (Hijau)
    if np.sum(lesion_masks['effusion']) > 0:
        vis[lesion_masks['effusion'] > 0] = (0, 255, 0) # Green

    # Calcification (Kuning)
    if np.sum(lesion_masks['calcification']) > 0:
        vis[lesion_masks['calcification'] > 0] = (0, 255, 255) # Yellow

    # Cavity (Merah)
    if np.sum(lesion_masks['cavity']) > 0:
        vis[lesion_masks['cavity'] > 0] = (0, 0, 255) # Red

    return vis

# Batch Running

input_root = "/kaggle/input/tuberculosis-tb-chest-xray-dataset"
output_vis_root = "lesion_segmentation_results"

categories = ['NORMAL', 'TUBERCULOSIS']

print(f"=== MULTI-LESION SEGMENTATION BATCH ===")

for category in categories:
    input_dir = os.path.join(input_root, category)
    save_dir = os.path.join(output_vis_root, category)
    os.makedirs(save_dir, exist_ok=True)

    # Ambil list gambar
    extensions = ['*.png', '*.jpg', '*.jpeg', '*.PNG']
    image_files = []
    for ext in extensions:
        image_files.extend(glob.glob(os.path.join(input_dir, ext)))

    print(f"\nProcessing {category} ({len(image_files)} images)...")

    for img_path in tqdm(image_files):
        try:
            filename = os.path.basename(img_path)

            # 1. Load & Segmentasi Paru
            img_original = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            if img_original is None: continue

            mask_lung, img_enhanced = segment_lungs(img_original)

            if mask_lung is not None:
                # 2. Deteksi Lesi
                lesion_masks = detect_lesions(img_enhanced, mask_lung)

                # 3. Visualisasi
                img_disp = cv2.resize(img_original, (512, 512))
                final_vis = create_multi_overlay(img_disp, mask_lung, lesion_masks)

                # Simpan
                cv2.imwrite(os.path.join(save_dir, filename), final_vis)

        except Exception as e:
            continue

print("\nSelesai! Cek folder:", output_vis_root)

Ekstraksi Fitur

In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
import glob
from tqdm import tqdm
from scipy.stats import skew, kurtosis
from skimage.feature import graycomatrix, graycoprops, local_binary_pattern, hog
from skimage.measure import regionprops

# Fungsi Ekstraksi Fitur
def extract_intensity_features(img, mask):
    """
    Mengambil statistik intensitas piksel HANYA di dalam area paru.
    Berguna untuk: Calcification (Putih Terang), Consolidation (Abu Terang).
    """
    # Ambil piksel di dalam mask
    roi_pixels = img[mask > 0]

    if len(roi_pixels) == 0:
        return [0] * 6 # Return 0 jika mask kosong

    # Statistik Dasar
    mean_val = np.mean(roi_pixels)
    std_val = np.std(roi_pixels)
    max_val = np.max(roi_pixels)
    min_val = np.min(roi_pixels)

    # Statistik Distribusi (Penting untuk Kalsifikasi)
    skewness_val = skew(roi_pixels) # Ekor distribusi (apakah ada sedikit piksel sangat putih?)
    kurtosis_val = kurtosis(roi_pixels) # Keruncingan histogram

    return [mean_val, std_val, max_val, min_val, skewness_val, kurtosis_val]

def extract_glcm_features(img, mask):
    """
    Mengambil fitur tekstur (Kasar/Halus).
    Berguna untuk: Infiltrate (Kasar), Fibrotic (Berserabut).
    """
    # GLCM butuh integer (uint8). Kita quantize ke 64 level biar cepat & robust noise.
    img_u8 = (img * 255).astype(np.uint8)

    # Masking: Area luar paru kita set ke 0.
    # Agar GLCM tidak menghitung tekstur background, kita potong bounding box paru saja.
    rows, cols = np.where(mask > 0)
    if len(rows) == 0: return [0] * 5

    y1, y2 = np.min(rows), np.max(rows)
    x1, x2 = np.min(cols), np.max(cols)

    # Crop ke ROI paru
    roi_img = img_u8[y1:y2, x1:x2]

    # Hitung GLCM (Jarak 1 piksel, 4 arah sudut: 0, 45, 90, 135 derajat)
    glcm = graycomatrix(roi_img, distances=[1], angles=[0, np.pi/4, np.pi/2, 3*np.pi/4],
                        levels=256, symmetric=True, normed=True)

    # Properti GLCM (Rata-rata dari 4 sudut)
    contrast = np.mean(graycoprops(glcm, 'contrast'))
    dissimilarity = np.mean(graycoprops(glcm, 'dissimilarity'))
    homogeneity = np.mean(graycoprops(glcm, 'homogeneity'))
    energy = np.mean(graycoprops(glcm, 'energy'))
    correlation = np.mean(graycoprops(glcm, 'correlation'))

    return [contrast, dissimilarity, homogeneity, energy, correlation]

def extract_lbp_features(img, mask):
    """
    Local Binary Pattern: Menangkap pola mikro-tekstur.
    Berguna untuk: Fibrosis detail.
    """
    img_u8 = (img * 255).astype(np.uint8)

    # Parameter LBP standar
    radius = 1
    n_points = 8 * radius

    # Hitung LBP
    lbp = local_binary_pattern(img_u8, n_points, radius, method='uniform')

    # Ambil histogram LBP hanya di dalam mask
    lbp_roi = lbp[mask > 0]

    if len(lbp_roi) == 0: return [0] * 10

    # Histogram LBP (n_points + 2 bins untuk metode uniform)
    n_bins = int(lbp_roi.max() + 1)
    hist, _ = np.histogram(lbp_roi, bins=n_bins, range=(0, n_bins), density=True)

    # Kita ambil 10 bin pertama sebagai fitur (atau statistik dari histogram)
    # Agar vektor fitur konsisten panjangnya, kita ambil statistik histogramnya
    # Atau ambil 10 bin pertama (uniform patterns)
    features = list(hist[:10])

    # Padding jika bin kurang dari 10
    if len(features) < 10:
        features += [0] * (10 - len(features))

    return features

def extract_shape_features(mask):
    """
    Mengambil geometri bentuk paru.
    Berguna untuk: Effusion (Sudut tumpul mengurangi Solidity), Collapse.
    """
    # Labeling (bisa jadi ada 2 paru terpisah)
    lbl_mask = measure.label(mask)
    props = regionprops(lbl_mask)

    if not props: return [0] * 5

    # Ambil properti dari region terbesar (atau rata-rata 2 terbesar)
    # Di sini kita ambil total area dan rata-rata properti bentuk
    total_area = 0
    solidity_list = []
    extent_list = []
    eccentricity_list = []
    perimeter_list = []

    for prop in props:
        total_area += prop.area
        solidity_list.append(prop.solidity)       # Kekompakan (Convexity)
        extent_list.append(prop.extent)           # Rasio area terhadap bounding box
        eccentricity_list.append(prop.eccentricity) # Kelonjongan
        perimeter_list.append(prop.perimeter)

    # Agregasi (Weighted Average berdasarkan area bisa lebih akurat, tapi Mean simpel cukup)
    feat_solidity = np.mean(solidity_list)
    feat_extent = np.mean(extent_list)
    feat_eccentricity = np.mean(eccentricity_list)

    # Compactness (Roundness): 4*pi*Area / Perimeter^2
    total_perimeter = np.sum(perimeter_list)
    if total_perimeter == 0: feat_compactness = 0
    else: feat_compactness = (4 * np.pi * total_area) / (total_perimeter ** 2)

    return [total_area, feat_solidity, feat_extent, feat_eccentricity, feat_compactness]

def extract_hog_features(img, mask):
    """
    HOG menangkap arah gradien (tepi).
    Berguna untuk: Cavity (Lingkaran) dan Infiltrate Edge.
    """
    img_resized = cv2.resize(img, (128, 128)) # Resize kecil untuk HOG global

    # Hitung HOG
    fd = hog(img_resized, orientations=8, pixels_per_cell=(16, 16),
             cells_per_block=(1, 1), visualize=False)

    # HOG menghasilkan ribuan fitur. Untuk SVM standar (bukan Deep Learning),
    # ringkas menjadi statistik HOG agar tidak overfitting.
    hog_mean = np.mean(fd)
    hog_std = np.std(fd)
    hog_max = np.max(fd)
    hog_kurtosis = kurtosis(fd)

    return [hog_mean, hog_std, hog_max, hog_kurtosis]

# BATCH EXTRACTION PIPELINE
def process_dataset(image_dir, mask_dir, categories):

    data = []

    print("Mulai Ekstraksi Fitur...")

    for category in categories:
        img_folder = os.path.join(image_dir, category)
        msk_folder = os.path.join(mask_dir, category)

        # Ambil list file
        image_files = glob.glob(os.path.join(img_folder, "*.*"))
        image_files = [f for f in image_files if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

        print(f"\nProcessing Class: {category} ({len(image_files)} images)")

        for img_path in tqdm(image_files):
            try:
                filename = os.path.basename(img_path)
                mask_name = os.path.splitext(filename)[0] + ".png" # Mask selalu PNG
                mask_path = os.path.join(msk_folder, mask_name)

                # 1. Load Image & Mask
                img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
                mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

                if img is None: continue

                # Jika mask tidak ditemukan (gagal segmentasi), skip atau handle
                if mask is None:
                    # print(f"Warning: Mask not found for {filename}")
                    continue

                # Resize image agar sama dengan mask (512x512 dari proses sebelumnya)
                img = cv2.resize(img, (512, 512))
                # Pastikan mask binary
                _, mask = cv2.threshold(mask, 127, 255, cv2.THRESH_BINARY)

                # Normalize Image (0-1 float)
                img_float = img.astype(np.float32) / 255.0

                # 2. EXTRACT FEATURES
                # A. Intensity (6)
                f_intensity = extract_intensity_features(img_float, mask)

                # B. GLCM Texture (5)
                f_glcm = extract_glcm_features(img_float, mask)

                # C. LBP Texture (10)
                f_lbp = extract_lbp_features(img_float, mask)

                # D. Shape (5)
                f_shape = extract_shape_features(mask)

                # E. HOG (4) - Kita pakai img_float yang di mask
                # Masking image untuk HOG (background jadi hitam)
                img_masked = img_float.copy()
                img_masked[mask == 0] = 0
                f_hog = extract_hog_features(img_masked, mask)

                # 3. COMBINE
                # Label: 0 untuk Normal, 1 untuk Tuberculosis
                label = 0 if category == "Normal" else 1

                row = [filename, label] + f_intensity + f_glcm + f_lbp + f_shape + f_hog
                data.append(row)

            except Exception as e:
                print(f"Error extracting {filename}: {e}")
                continue

    return data

# --- SETUP PATH ---
dataset_images_path = "../tb-chest-xray-classifier/datasets/tawsifurrahman/tuberculosis-tb-chest-xray-dataset/versions/3/TB_Chest_Radiography_Database"          # Folder: Normal, Tuberculosis
dataset_masks_path = "masks_v26"

categories = ['Normal', 'Tuberculosis']

# --- RUN ---
extracted_data = process_dataset(dataset_images_path, dataset_masks_path, categories)

# --- SAVE TO CSV ---
# Definisikan nama kolom
columns = ['filename', 'label']
columns += ['mean', 'std', 'max', 'min', 'skew', 'kurt'] # Intensity
columns += ['contrast', 'dissimilarity', 'homogeneity', 'energy', 'correlation'] # GLCM
columns += [f'lbp_{i}' for i in range(10)] # LBP
columns += ['area', 'solidity', 'extent', 'eccentricity', 'compactness'] # Shape
columns += ['hog_mean', 'hog_std', 'hog_max', 'hog_kurt'] # HOG

df = pd.DataFrame(extracted_data, columns=columns)

# Simpan
output_csv = "cxr_tb_features_v1.csv"
df.to_csv(output_csv, index=False)

print(f"\nSelesai! Fitur tersimpan di {output_csv}")
print(f"Total Data: {len(df)}")
print(df.head())

Training

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
# Tambahkan precision_score, recall_score, f1_score di sini
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc, accuracy_score, f1_score, precision_score, recall_score
from sklearn.impute import SimpleImputer
import joblib

# PERSIAPAN DATA

# Load Data
csv_path = "cxr_tb_features_v1.csv"
print(f"Loading data from {csv_path}...")
df = pd.read_csv(csv_path)

# Cek data kosong/NaN
print(f"Jumlah NaN sebelum cleaning: {df.isnull().sum().sum()}")
df = df.dropna()

# Pisahkan Fitur (X) dan Target (y)
X = df.drop(['filename', 'label'], axis=1)
y = df['label'] # 0 = Normal, 1 = Tuberculosis

print(f"\nDistribusi Kelas:")
print(y.value_counts())

# Split Data (80% Train, 20% Test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# PIPELINE PRAPEMROSESAN

# A. Imputasi
imputer = SimpleImputer(strategy='mean')
X_train = imputer.fit_transform(X_train)
X_test = imputer.transform(X_test)

# B. Scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# TRAINING DENGAN GRID SEARCH

print("\nMemulai Hyperparameter Tuning (Mencari setting terbaik)...")

# Definisi Parameter
param_grid = {
    'C': [0.1, 1, 10, 100],
    'gamma': ['scale', 'auto', 0.1, 0.01],
    'kernel': ['rbf', 'linear']
}

# Inisialisasi SVM dengan class_weight balanced
svm = SVC(class_weight='balanced', probability=True, random_state=42)

# Grid Search (Optimasi berdasarkan F1-Macro agar adil untuk kedua kelas)
grid = GridSearchCV(svm, param_grid, refit=True, verbose=2, cv=5, scoring='f1_macro')
grid.fit(X_train_scaled, y_train)

print(f"\nParameter Terbaik: {grid.best_params_}")
best_model = grid.best_estimator_

# EVALUASI LENGKAP

print("\nEvaluasi pada Test Set...")
y_pred = best_model.predict(X_test_scaled)
y_prob = best_model.predict_proba(X_test_scaled)[:, 1]

# --- HITUNG METRIK UTAMA SECARA EKSPLISIT ---
acc = accuracy_score(y_test, y_pred)
# Kita fokus pada kelas 1 (Tuberculosis) untuk F1 Score
f1 = f1_score(y_test, y_pred, pos_label=1)
prec = precision_score(y_test, y_pred, pos_label=1)
rec = recall_score(y_test, y_pred, pos_label=1)

print("\n" + "="*30)
print("HASIL PERFORMA MODEL")
print("="*30)
print(f"Akurasi (Accuracy)  : {acc*100:.2f}%")
print(f"F1-Score (TB Class) : {f1*100:.2f}%")
print(f"Precision (TB Class): {prec*100:.2f}%")
print(f"Recall (TB Class)   : {rec*100:.2f}%")
print("="*30)

# A. Classification Report (Detail per kelas)
print("\n--- Detailed Classification Report ---")
print(classification_report(y_test, y_pred, target_names=['Normal', 'Tuberculosis']))

# B. Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(6, 5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Normal', 'TB'], yticklabels=['Normal', 'TB'])
plt.xlabel('Prediksi Model')
plt.ylabel('Kenyataan (Ground Truth)')
plt.title(f'Confusion Matrix\nF1 Score: {f1:.2f}') # Menampilkan F1 di Judul Plot
plt.show()

# C. ROC Curve
fpr, tpr, thresholds = roc_curve(y_test, y_prob)
roc_auc = auc(fpr, tpr)

plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC)')
plt.legend(loc="lower right")
plt.show()

# SIMPAN MODEL
joblib.dump(best_model, 'svm_tb_model.pkl')
joblib.dump(scaler, 'scaler.pkl')
print("\nModel dan Scaler telah disimpan!")

Testing

In [ ]:
import matplotlib.pyplot as plt
import joblib
import cv2
import numpy as np
import os
import glob
from skimage import exposure

# LOAD MODEL
try:
    scaler = joblib.load('scaler.pkl')
    model = joblib.load('svm_tb_model.pkl')
    print("✅ Model & Scaler berhasil dimuat.")
except:
    print("❌ Gagal memuat model. Pastikan file .pkl ada di folder ini.")

# FUNGSI HITUNG PERSENTASE LESI
def calculate_lesion_stats(lung_mask, lesion_masks):
    """
    Menghitung berapa persen area paru yang tertutup oleh setiap jenis lesi.
    """
    total_lung_area = np.sum(lung_mask > 0)
    stats = {}

    if total_lung_area == 0:
        return {k: 0.0 for k in lesion_masks.keys()}

    for lesion_name, l_mask in lesion_masks.items():
        lesion_area = np.sum(l_mask > 0)
        percentage = (lesion_area / total_lung_area) * 100
        stats[lesion_name] = percentage

    return stats

# FUNGSI UTAMA TESTING
def test_and_visualize(image_path):
    # A. Load & Preprocess
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        print(f"Gagal membaca {image_path}")
        return

    # Resize & Float
    img = cv2.resize(img, (512, 512))
    img_float = img.astype(np.float32) / 255.0

    # B. Segmentasi Paru (V26)
    mask_lung, img_enhanced = segment_lungs(img)

    if mask_lung is None:
        print("Segmentasi Paru Gagal.")
        return

    # Normalize mask
    _, mask_bin = cv2.threshold(mask_lung, 127, 255, cv2.THRESH_BINARY)

    # C. Ekstraksi Fitur -> urutannya harus sama dengan training
    f_int = extract_intensity_features(img_float, mask_bin)
    f_glcm = extract_glcm_features(img_float, mask_bin)
    f_lbp = extract_lbp_features(img_float, mask_bin)
    f_shp = extract_shape_features(mask_bin)

    img_masked = img_float.copy()
    img_masked[mask_bin == 0] = 0
    f_hog = extract_hog_features(img_masked, mask_bin)

    features = f_int + f_glcm + f_lbp + f_shp + f_hog
    features_vector = np.array(features).reshape(1, -1)

    # D. Prediksi SVM
    features_scaled = scaler.transform(features_vector)
    prediction = model.predict(features_scaled)[0] # 0=Normal, 1=TB
    proba = model.predict_proba(features_scaled)[0] # [Prob_Normal, Prob_TB]

    confidence = proba[prediction] * 100
    label_str = "Tuberculosis" if prediction == 1 else "Normal"
    color_code = 'red' if prediction == 1 else 'green'

    # E. Deteksi Lesi & Overlay (Hanya jika TB)
    lesion_masks = None
    lesion_stats = {}

    if prediction == 1:
        # Deteksi Lesi (Strict Mode)
        lesion_masks = detect_lesions(img_enhanced, mask_bin)
        # Hitung Persentase
        lesion_stats = calculate_lesion_stats(mask_bin, lesion_masks)
        # Buat Overlay Warna-warni
        overlay_img = create_multi_overlay(img, mask_bin, lesion_masks)
    else:
        # Jika Normal, hanya overlay ungu transparan (Paru sehat)
        overlay_img = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
        lung_color = (255, 0, 255)
        temp = overlay_img.copy()
        temp[mask_bin > 0] = lung_color
        overlay_img = cv2.addWeighted(temp, 0.3, overlay_img, 0.7, 0)

    # F. VISUALISASI MATPLOTLIB
    plt.figure(figsize=(14, 6))

    # Plot 1: Asli
    plt.subplot(1, 2, 1)
    plt.imshow(img, cmap='gray')
    plt.title(f"Citra Asli\n({os.path.basename(image_path)})")
    plt.axis('off')

    # Plot 2: Hasil AI
    plt.subplot(1, 2, 2)
    # Convert BGR (OpenCV) to RGB (Matplotlib)
    plt.imshow(cv2.cvtColor(overlay_img, cv2.COLOR_BGR2RGB))
    plt.title(f"Prediksi: {label_str}\nConfidence: {confidence:.2f}%",
              color=color_code, fontweight='bold', fontsize=14)
    plt.axis('off')

    # Tampilkan Statistik Lesi di bawah gambar
    if prediction == 1:
        stat_text = (
            f"DETEKSI LESI (Persentase Area Paru):\n"
            f"🔵 Infiltrate/Fibrosis : {lesion_stats.get('infiltrate', 0):.2f}%\n"
            f"🔴 Cavity (Lubang)     : {lesion_stats.get('cavity', 0):.2f}%\n"
            f"🟡 Calcification       : {lesion_stats.get('calcification', 0):.2f}%\n"
            f"🟢 Effusion (Cairan)   : {lesion_stats.get('effusion', 0):.2f}%"
        )
        plt.figtext(0.5, 0.01, stat_text, ha="center", fontsize=12,
                    bbox={"facecolor":"white", "alpha":0.8, "pad":5})
    else:
        plt.figtext(0.5, 0.05, "✅ Paru-paru Normal. Tidak ada lesi signifikan terdeteksi.",
                    ha="center", fontsize=12, color='green',
                    bbox={"facecolor":"white", "alpha":0.8, "pad":5})

    plt.tight_layout()
    plt.show()

# EKSEKUSI PADA SAMPEL TESTING
# Ganti list ini dengan path file testing
test_images = [
    "Dataset/Tuberculosis/Tuberculosis-10.png", # Ganti dengan path file yang benar
    "Dataset/Normal/Normal-15.png",
    # Tambahkan file lain...
]

# Atau scan folder testing otomatis
# test_images = glob.glob("Dataset/Test_Set/*.png")[:5]

print(f"Memulai Testing pada {len(test_images)} gambar...\n")

for img_path in test_images:
    if os.path.exists(img_path):
        test_and_visualize(img_path)
    else:
        print(f"File tidak ditemukan: {img_path}")